Inspired by https://www.kaggle.com/code/ghalebdweikat/genetic-algorithm-tutorial

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_machines = pd.read_csv('tsi_data/dataset_2.csv')
df_sites = pd.read_csv('tsi_data/scenario_2.csv')

In [3]:
df_machines

,machine,inventory,time,productivity
0,A1,4,1.0,1225
1,A2,5,1.5,1575
2,A3,1,2.5,2475
3,A4,7,2.5,1750
4,A5,3,3.0,1750
5,A6,10,3.5,3150
6,A7,7,2.5,2700
7,A8,1,2.5,3150
8,A9,9,3.5,3150
9,A10,10,4.0,3825


In [4]:
df_sites

,site,cleaning_area,cleaning_time
0,S1,14300,5
1,S2,82000,4
2,S3,87000,4
3,S4,21000,3
4,S5,27900,8
5,S6,61000,4


### Representation

We will take, for one site, the chromosome of length nb_machines, with 1 if the machine is used and 0 if it is not.

Example to clean a site with A1 and A2 :
[ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ] 

### Generating random chromosomes / solutions

In [5]:
def generateChromosome(machines, max_cleaning_time):
    """
    Returns a chromosome of size nb_machines, = to an array of True (machine used) and False (machine not used)
    The only constraint is that the clean time of the machine can't be higher than allowed by the site
    """
    # Get the indexes of machines with a valid cleaning time (inferior to the cleaning time required by the site)
    allowed_machines_indexes = machines.index[machines['time'] <= max_cleaning_time].to_numpy()
    
    # Randomly select some machines
    selected_machines_indexes = [idx for idx in allowed_machines_indexes if np.random.randint(0, 2)]
    
    # For each machine True if machine_index is in selected list, False otherwise
    return np.isin(machines.index, selected_machines_indexes)  

In [6]:
# Example: Select random machines for a max cleaning time of 3h
print(generateChromosome(df_machines, 3).astype(int))

[1 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0]


In [7]:
def generateParents(population_size, machines, max_cleaning_time):
    """
    Returns an array with a solution by site
    """
    return [generateChromosome(machines, max_cleaning_time) for _ in range(population_size)]

In [8]:
# Example: A generation of 3 chromosomes, for a max cleaning time of 2h30
for x in generateParents(3, df_machines, 2.5):
    print(x.astype(int))

[1 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]


### Getting total cleaned area for one chromosome

In [9]:
def totalCleanedArea(chromosome, machines):
    return machines["productivity"][chromosome].sum()

In [10]:
# Example with a random solution
chromosome = generateChromosome(df_machines, 3)
print("Chromosome:", chromosome.astype(int))
print("Total cleaned area: {}m²".format(totalCleanedArea(chromosome, df_machines)))

Chromosome: [1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Total cleaned area: 10220m²


### Getting indexes of selected machines

In [11]:
def selectedMachinesIndexes(chromosome):
    return np.where(chromosome)[0]

In [12]:
# Example with a random solution
chromosome = generateChromosome(df_machines, 3)
print("Chromosome:", chromosome.astype(int))
print("Indexes of machines selected:", ', '.join(selectedMachinesIndexes(chromosome).astype(str)))

Chromosome: [0 1 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0]
Indexes of machines selected: 1, 4, 6, 7, 11, 16


### Constraint

In [13]:
def reduceMaxCleaningTime(chromosome, machines, max_cleaning_time):
    """
    Apply the constraint by removing from the chromosome the machines that have a cleaning time too high
    """
    chromosome[machines["time"] > max_cleaning_time] = False
    return chromosome

In [14]:
chromosome = generateChromosome(df_machines, 3)
print("Chromosome:", chromosome.astype(int))
print("Modified:  ", reduceMaxCleaningTime(chromosome, df_machines, 2.5).astype(int))

Chromosome: [0 0 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
Modified:   [0 0 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]


### Mutations

In [15]:
def mutate(chromosome):
    """
    Mutate a random item of solution (0→1 or 1→0)
    """
    rand_index = np.random.randint(len(chromosome))
    chromosome[rand_index] = not chromosome[rand_index]
    return chromosome

In [16]:
chromosome = generateChromosome(df_machines, 3)
print("Chromosome:", chromosome.astype(int))
mutate(chromosome)
print("Mutated:   ", chromosome.astype(int))

Chromosome: [0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0]
Mutated:    [0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]


### Update population

In [17]:
def updatePopulation(generation, population_size, machines, cleaning_area, max_cleaning_time):
    """
    Fix chromosomes with a higher cleaning time that allowed
    Compute fitness of each chromosome and return the best ones
    """
    for chromosome in generation:
        # Unselect machines with a higher cleaning time that allowed
        chromosome = reduceMaxCleaningTime(chromosome, machines, max_cleaning_time)
    
        # Unselect machines to reduce the total cleaned size
        while(totalCleanedArea(chromosome, machines) > cleaning_area):
            # Unselect a random machine
            random_selected_machine_index = np.random.choice(selectedMachinesIndexes(chromosome))
            chromosome[random_selected_machine_index] = False
            
            # If we just cross the treshold, add it back and exit to clean a little bit more than needed
            if (totalCleanedArea(chromosome, machines) <= cleaning_area):
                chromosome[random_selected_machine_index] = True
                break
                
    
    fitness = np.array([len(selectedMachinesIndexes(chromosome)) for chromosome in generation])
    
    return [generation[i] for i in np.argsort(fitness)][:population_size]

### Crossover

In [18]:
def crossover(first_chromosome, second_chromosome):
    """
    Takes two parent chromosomes and returns two child chromosomes
    The first chromosome takes half of his digits in the first parent and the other half in the second parent
    The second chromosome takes the opposite halves.
    """
    n = len(first_chromosome)
    first_half = np.random.choice(n, round(n/2), False)
    
    first_child = [first_chromosome[i] if i in first_half else second_chromosome[i] for i in range(n)]
    second_child = [second_chromosome[i] if i in first_half else first_chromosome[i] for i in range(n)]
    return mutate(first_child), mutate(second_child)

In [19]:
crossover ([False, True, False, True], [False, True, True, True])

([False, False, True, True], [False, True, False, False])

### New Generation

In [20]:
def newGeneration(generation):
    """
    Keep the top 2 (already sorted by fitness), and build need
    child by crossover on all pairs of the top 4
    """
    top4 = generation[:4]
    
    new_gen = [generation[:2]]
    for i in range (4):
        for j in range (4):
            if i != j:
                childs = crossover(top4[i], top4[j])
                new_gen.append(childs[0])
                new_gen.append(childs[1])
                
    return new_gen

## Optimize

### Use Case 2

In [21]:
df_machines = pd.read_csv('tsi_data/dataset_2.csv')

### Flatten dataframe

To facilitate the process, we will convert our dataframe to have as many rows as machine.

In [22]:
def flattenDataframe(df):
    """
    Repeat the row for each machine given its inventory
    """
    return df.loc[df.index.repeat(df['inventory'])].drop('inventory', axis=1).reset_index(drop=True)

In [23]:
df_machines = flattenDataframe(df_machines)
df_machines

,machine,time,productivity
0,A1,1.0,1225
1,A1,1.0,1225
2,A1,1.0,1225
3,A1,1.0,1225
4,A2,1.5,1575
...,...,...,...
122,D1,3.0,7100
123,D1,3.0,7100
124,D2,6.0,4250
125,D2,6.0,4250


In [24]:
results = pd.DataFrame(columns=["Site", "Machines selected", "Cleaned area", "Area to clean", "Total cleaning time", "Allowed cleaning time"])

def optimize(machines, sites, population_size, generationsPerSite):
    current_site_idx = 0
    while(True):
        if(len(machines) == 0 or current_site_idx >= len(sites)):
            break
        
        current_site = sites.iloc[current_site_idx]
        max_cleaning_time = current_site["cleaning_time"]
        cleaning_area = current_site["cleaning_area"]
    
        parents = generateParents(population_size, machines, max_cleaning_time)
        generation = updatePopulation(parents, population_size, machines, cleaning_area, max_cleaning_time)
        new_generation = generation
        for i in range(generationsPerSite):
            new_generation = newGeneration(new_generation)
            new_generation = updatePopulation(parents, population_size, machines, cleaning_area, max_cleaning_time)
        
        # Save current results in dataframe
        best_chromosome = new_generation[0]
        selected_machines = machines[best_chromosome]
        selected_machines_names = selected_machines['machine'].values
        current_cleaned_area = totalCleanedArea(best_chromosome, machines)
        max_selected_cleaning_time = np.max(selected_machines['time'])
        results.loc[current_site_idx] = [
            current_site['site'], 
            str(selected_machines_names), 
            str(current_cleaned_area) + ' m²', 
            str(cleaning_area) + ' m²', 
            str(max_selected_cleaning_time) + 'h', 
            str(max_cleaning_time)  + 'h']
        
        # Remove selected machines
        machines = machines[[not x for x in new_generation[0]]]
        current_site_idx += 1
    
    print("Unused machines:", ", ".join(machines["machine"].values))
    
    return results

In [25]:
population_size = 20
generationsPerSite = 3
machines = df_machines.copy()

optimize(machines, df_sites, population_size, generationsPerSite)

Unused machines: A2, A2, A2, A3, A5, A6, A6, A6, A6, A6, A6, A7, A9, A9, A10, B1, B1, B3, B3, B4, B4, B4, B4, B5, B5, B5, B5, B5, B5, B5, B5, C1, C2, C3, C3, C4, C4, C4, D2, D2


,Site,Machines selected,Cleaned area,Area to clean,Total cleaning time,Allowed cleaning time
0,S1,['C4' 'C5'],16740 m²,14300 m²,5.0h,5h
1,S2,['A1' 'A1' 'A4' 'A7' 'A7' 'A7' 'A9' 'A9' 'A10'...,83830 m²,82000 m²,4.0h,4h
2,S3,['A2' 'A4' 'A4' 'A4' 'A5' 'A5' 'A6' 'A6' 'A8' ...,88180 m²,87000 m²,4.0h,4h
3,S4,['A2' 'A4' 'A4' 'A7' 'B2' 'B2' 'B2'],12935 m²,21000 m²,2.5h,3h
4,S5,['A6' 'A10' 'C4' 'C5' 'D2'],27965 m²,27900 m²,6.0h,8h
5,S6,['A1' 'A1' 'A4' 'A6' 'A7' 'A7' 'A10' 'B1' 'B1'...,38035 m²,61000 m²,4.0h,4h
